In [1]:
# IMPORTS
import os
import json
import time


from tqdm import tqdm
import pandas as pd

import requests


In [2]:
# SIGN_IN
url = "http://localhost:8000"
token = ""

# TODO: DO NOT PUSH THESE!!!!!!!!!!!!!!!!!!!!!!
sign_in = requests.post(
    url=f"{url}/users/authtoken/",
    data={
        "username": "",
        "password": ""
    }
)
if sign_in.status_code == 200:
    token = sign_in.json()["login"]["token"]
    headers = {"Authorization": f"Token {token}", "Content-type": "application/json"}

    print(sign_in.status_code)
else:
    print(f"Issue signing in\n {sign_in.json()}")

200


In [5]:
# POST NEW INDICATORS

with open('formatted_farmafrica_data_can_finally_move_on.json') as f:
    ###### UPLOAD LOCALLY ############
    new_indicators = json.load(f)
    for n_indicator in tqdm(new_indicators):
        n_indicator["user_id"] = ""
        # print(len(new_indicators), n_indicator)
        # break
        update_field = requests.post(
            url=f"{url}/layers/fieldindicators/",
            data=json.dumps(n_indicator), headers=headers
        )
    
    ###### UPLOAD ONLINE ##############
    # kator_dict = {}
    # for n_indicator in new_indicators:
    #     n_indicator["user_id"] = ""
    #     # print(n_indicator)
    #     kator_dict[f"{n_indicator['field_id']}{n_indicator['indicator']}"] = n_indicator
    # update_field = requests.post(
    #     url=f"{url}/layers/fieldindicators/",
    #     data=json.dumps(kator_dict), headers=headers
    # )
    ################################

100%|██████████| 56/56 [01:37<00:00,  1.74s/it]


In [22]:
# GET OLD INDICATORS

if sign_in.status_code == 200:
    existing_indicators = requests.get(f"{url}/layers/fieldindicators/", headers=headers)
    existing_indicators = existing_indicators.json()
    print(len(existing_indicators), "is the length of the old fields\n", type(existing_indicators))
else:
    print(f"Failed to sign in. Error is\n\n{sign_in.json()}")

0 is the length of the old fields
 <class 'list'>


In [8]:
# DELETE OLD INDICATOR CALCULATIONS AND ADD THE NEW ONES

# GET INDICATOR_CALCNS
existing_indicator_calcns = requests.get(f"{url}/layers/indicatorcalculations/", headers=headers)
existing_indicator_calcns = existing_indicator_calcns.json()
print(existing_indicator_calcns)
"""
select * from layers_fieldindicatorcalculations where user_id=''; --- TO SEE DEM

delete * from layers_fieldindicatorcalculations where user_id='';
"""

[{'crop_type': 'Tobacco', 'indicator': 'field_ndvi', 'january': 0.23, 'february': 0.21, 'march': 0.3, 'april': 0.28, 'may': 0.28, 'june': 0.26, 'july': 0.15, 'august': 0.14, 'september': 0.13, 'october': 0.12, 'november': 0.13, 'december': 0.13}, {'crop_type': 'Tobacco', 'indicator': 'field_ndwi', 'january': 0.37, 'february': 0.49, 'march': 0.36, 'april': 0.23, 'may': 0.16, 'june': 0.03, 'july': 0.05, 'august': -0.06, 'september': -0.1, 'october': -0.07, 'november': -0.11, 'december': 0.18}, {'crop_type': 'Tobacco', 'indicator': 'field_rainfall', 'january': 52.22, 'february': 36.09, 'march': 7.87, 'april': 3.48, 'may': 0.5, 'june': 0.96, 'july': 0.71, 'august': 0.2, 'september': 0.26, 'october': 3.64, 'november': 2.94, 'december': 35.65}, {'crop_type': 'Tobacco', 'indicator': 'field_temperature', 'january': 21.99, 'february': 22.28, 'march': 21.48, 'april': 21.9, 'may': 20.57, 'june': 17.67, 'july': 17.24, 'august': 20.27, 'september': 23.76, 'october': 26.97, 'november': 27.33, 'decem

"\nselect * from layers_fieldindicatorcalculations where user_id='5fdb02c45bcec9000a7d19aa'; --- TO SEE DEM\n\ndelete * from layers_fieldindicatorcalculations where user_id='5fdb02c45bcec9000a7d19aa';\n"

In [32]:
# POST NEW INDICATOR CALCULATIONS

# TODO: This needs to be done every month

indicators = ["field_ndvi", "field_ndwi", "field_rainfall", "field_temperature"]
if sign_in.status_code == 200:
    with open('new_oppo_field_calcns.json') as f:
        new_indicator_calcns = json.load(f)
        list_indi_calcs = []
        for kator in indicators:
            for crop in new_indicator_calcns[kator].keys():
                one_row = {**new_indicator_calcns[kator][crop]}
                one_row["indicator"] = kator
                one_row["crop_type"] = crop
                one_row["user_id"] = ""
                list_indi_calcs.append(one_row)

        for row in tqdm(list_indi_calcs):
            update_field = requests.post(
                url=f"{url}/layers/indicatorcalculations/",
                data=json.dumps(row), headers=headers
            )
            time.sleep(1)
else:
    print(f"Failed to sign in. Error is\n\n{sign_in.json()}")


100%|██████████| 4/4 [00:10<00:00,  2.63s/it]


In [14]:
# GENERATE FORECAST CSV

ab_df = pd.read_json("2021-05-05-abInBev.json")
trias_df = pd.read_json("2021-05-05-trias.json")

# print(len(ab_df.index))
# len(trias_df.index)
ab_df["user_id"] = "5fa113599e2a01000a655cee"
trias_df["user_id"] = "605d97e0e68e66000b8ea4be"
# trias_df.head()

two_dfs = pd.concat([ab_df, trias_df], ignore_index=True)



# REORDER the columns
# HARD TO DO IT FROM LIBRE OFFICE CALC
# ["id","field_id","day","user_id","avg_temperature","sum_precipitation"]
# two_dfs = two_dfs.reindex(columns=["id","field_id","day","user_id","temperatures.max","precipitation.amount"])
# len(two_dfs.index)
# two_dfs.tail()
two_dfs.to_csv("2021-05-05_trias_abinbev.csv")